In [ ]:
import socket
import time

from pop import Gas
from pop import Dust
from pop import TempHumi
from pop import Pir  # PIR이 pop 라이브러리에 포함되어 있음

# ---------------------------
# JAVA 서버 주소 & 포트
# ---------------------------
JAVA_IP = "192.168.101.102"   # 윈도우 IP
JAVA_PORT = 39187             # 네 서버 포트

# ---------------------------
# POP 센서 초기화
# ---------------------------
gas = Gas(6)
dust = Dust()
th = TempHumi()
pir = Pir(22)

# ---------------------------
# TCP 연결
# ---------------------------
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((JAVA_IP, JAVA_PORT))
print("[PY] JAVA 서버 연결 완료")

# ---------------------------
# 센서값 측정 + 전송 루프
# ---------------------------
while True:
    # ---- Gas ----
    gas_val = gas.readAverage()
    methan = gas.calcMethan(gas_val)

    # ---- Dust ----
    dust.read()
    pm1  = dust.pm_1p0_grimm
    pm25 = dust.pm_2p5_grimm
    pm10 = dust.pm_10_grimm

    # ---- Temp & Humi ----
    temp = th.getTemperature()
    humi = th.getHumidity()

    # ---- PIR ----
    pir_val = pir.read()
    # 1 = movement, 0 = no movement

    # ---- JAVA로 전송할 패킷 생성 ----
    message = (
        f"SENSOR "
        f"GAS={gas_val} "
        f"METHAN={methan} "
        f"TEMP={temp:.2f} "
        f"HUMI={humi:.2f} "
        f"PM1={pm1} "
        f"PM25={pm25} "
        f"PM10={pm10} "
        f"PIR={pir_val}"
    )

    try:
        sock.sendall((message + "\n").encode())
    except:
        print("[PY] 서버 연결 끊김. 재연결 시도 중...")
        time.sleep()
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((JAVA_IP, JAVA_PORT))
        continue

    time.sleep(1)   # 1초마다 전송
